# **Bước 0: Import thư viện**

In [1]:
import keras
import os
import glob
import json
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import random
import shutil
from tensorflow.keras import models, layers,Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# **Bước 1: Đọc ảnh và file JSON**

In [2]:
# Đường dẫn thư mục dữ liệu gốc
IMAGE_DIR = "/kaggle/input/btxrd-data/BTXRD/BTXRD/images"
JSON_DIR = "/kaggle/input/btxrd-data/BTXRD/BTXRD/Annotations"

# Thư mục lưu kết quả sau khi xử lý
OUTPUT_IMAGE_DIR = "/kaggle/working/processed_images"
OUTPUT_JSON_DIR = "/kaggle/working/processed_annotations"

if not os.path.exists(OUTPUT_IMAGE_DIR):
    os.makedirs(OUTPUT_IMAGE_DIR, exist_ok=True)  # Tạo thư mục nếu chưa có
    print(f"✅ Đã tạo thư mục: {OUTPUT_IMAGE_DIR}")
else:
    print(f"✅ Thư mục đã tồn tại: {OUTPUT_IMAGE_DIR}")

if not os.path.exists(OUTPUT_JSON_DIR):
    os.makedirs(OUTPUT_JSON_DIR, exist_ok=True)  # Tạo thư mục nếu chưa có
    print(f"✅ Đã tạo thư mục: {OUTPUT_JSON_DIR}")
else:
    print(f"✅ Thư mục đã tồn tại: {OUTPUT_JSON_DIR}")

✅ Đã tạo thư mục: /kaggle/working/processed_images
✅ Đã tạo thư mục: /kaggle/working/processed_annotations


# ** Bước 2: Tiền xử lý dữ liệu**

Resize ảnh về kích thước cố định nhưng không méo hình.  
Padding ảnh để giữ nguyên tỷ lệ.  
Cập nhật tọa độ vùng ung thư trong JSON để không bị sai lệch.  

In [3]:
def resize_and_pad_image(img_path, json_path):
    """ Resize ảnh về kích thước chuẩn và cập nhật tọa độ vùng ung thư trong JSON. """
    # Đọc ảnh gốc
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    h, w = img.shape

    # Tính tỷ lệ resize để không làm méo ảnh
    scale = min(TARGET_SIZE[0] / w, TARGET_SIZE[1] / h)
    new_w, new_h = int(w * scale), int(h * scale)
    
    # Resize ảnh
    resized_img = cv2.resize(img, (new_w, new_h))

    # Tạo ảnh nền trắng để pad
    final_img = np.ones((TARGET_SIZE[1], TARGET_SIZE[0]), dtype=np.float32) * 255
    x_offset = (TARGET_SIZE[0] - new_w) // 2
    y_offset = (TARGET_SIZE[1] - new_h) // 2

    # Đưa ảnh vào trung tâm
    final_img[y_offset:y_offset + new_h, x_offset:x_offset + new_w] = resized_img

    # Cập nhật JSON
    with open(json_path, "r") as f:
        data = json.load(f)

    # Cập nhật tọa độ vùng tổn thương
    for shape in data["shapes"]:
        updated_points = []
        for point in shape["points"]:
            x_new = point[0] * scale + x_offset
            y_new = point[1] * scale + y_offset
            updated_points.append([x_new, y_new])

        shape["points"] = updated_points

    return final_img / 255.0, data

In [4]:
TARGET_SIZE = (224, 294)  # (width, height)

# Duyệt qua tất cả ảnh trong thư mục
image_paths = glob.glob(IMAGE_DIR + "/*.png") + glob.glob(IMAGE_DIR + "/*.jpg") + glob.glob(IMAGE_DIR + "/*.jpeg")

for img_path in image_paths:
    filename = os.path.basename(img_path).split('.')[0]  # Lấy tên file không có đuôi
    json_path = os.path.join(JSON_DIR, filename + ".json")  # Tìm file JSON tương ứng
    
    if os.path.exists(json_path):
        # Xử lý ảnh + tọa độ ung thư
        processed_img, updated_annotations = resize_and_pad_image(img_path, json_path)

        if processed_img is not None:
            # Lưu ảnh mới
            output_img_path = os.path.join(OUTPUT_IMAGE_DIR, filename + ".jpg")
            cv2.imwrite(output_img_path, (processed_img * 255).astype(np.uint8))  # Chuyển về 0-255

            # Lưu JSON mới
            output_json_path = os.path.join(OUTPUT_JSON_DIR, filename + ".json")
            with open(output_json_path, "w") as f:
                json.dump(updated_annotations, f, indent=4)
    else:
        # Xử lý ảnh không có JSON
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        processed_img = cv2.resize(img, TARGET_SIZE)  # Resize ảnh về đúng kích thước

        # Lưu ảnh đã resize vào thư mục OUTPUT_IMAGE_DIR
        output_img_path = os.path.join(OUTPUT_IMAGE_DIR, filename + ".jpg")
        cv2.imwrite(output_img_path, processed_img)

print("✅ Xử lý hoàn tất. Ảnh và JSON đã được lưu tại", OUTPUT_IMAGE_DIR, "và", OUTPUT_JSON_DIR)


✅ Xử lý hoàn tất. Ảnh và JSON đã được lưu tại /kaggle/working/processed_images và /kaggle/working/processed_annotations


# **Bước 3: Phân chia bộ test và train**

Phân chia ảnh thành 80% train và 20% test

In [5]:
OUTPUT_DIR = "/kaggle/working/"
# Tạo thư mục train, test, val
for folder in ["train", "test", "val"]:
    os.makedirs(os.path.join(OUTPUT_DIR, folder), exist_ok=True)

# Lấy danh sách ảnh
image_paths = glob.glob(os.path.join(OUTPUT_IMAGE_DIR, "*.jpg")) + \
              glob.glob(os.path.join(OUTPUT_IMAGE_DIR, "*.jpeg"))

# Shuffle dữ liệu để chia ngẫu nhiên
random.seed(42)
random.shuffle(image_paths)

# Chia theo tỷ lệ 70% train - 20% test - 10% validation
train_idx = int(0.7 * len(image_paths))
test_idx = int(0.9 * len(image_paths))  # Sau train, còn 30%, chia tiếp thành 20% test + 10% val

train_images = image_paths[:train_idx]
test_images = image_paths[train_idx:test_idx]
val_images = image_paths[test_idx:]

# Di chuyển ảnh vào thư mục tương ứng
for img_path in train_images:
    shutil.copy(img_path, os.path.join(OUTPUT_DIR, "train", os.path.basename(img_path)))

for img_path in test_images:
    shutil.copy(img_path, os.path.join(OUTPUT_DIR, "test", os.path.basename(img_path)))

for img_path in val_images:
    shutil.copy(img_path, os.path.join(OUTPUT_DIR, "val", os.path.basename(img_path)))

print("✅ Chia tập dữ liệu hoàn tất!")

✅ Chia tập dữ liệu hoàn tất!


In [6]:
import os

test_path = "/kaggle/working/test" 
test_files = len(os.listdir(test_path))
print(f"📂 Số lượng file trong {test_path}: {test_files}")

benign_path = "/kaggle/working/train" 
benign_files = len(os.listdir(benign_path)) 
print(f"📂 Số lượng file trong {benign_path}: {benign_files}")

malignant_path = "/kaggle/working/val"
malignant_files = len(os.listdir(malignant_path)) 
print(f"📂 Số lượng file trong {malignant_path}: {malignant_files}")

📂 Số lượng file trong /kaggle/working/test: 749
📂 Số lượng file trong /kaggle/working/train: 2622
📂 Số lượng file trong /kaggle/working/val: 375


In [7]:
# Thư mục train gốc
TRAIN_DIR = "/kaggle/working/train"
OUTPUT_JSON_DIR = "/kaggle/working/processed_annotations"

# Tạo thư mục benign/malignant trong train
for folder in ["benign", "malignant"]:
    os.makedirs(os.path.join(TRAIN_DIR, folder), exist_ok=True)

# Lấy danh sách ảnh trong train
train_images = glob.glob(os.path.join(TRAIN_DIR, "*.jpg")) + \
               glob.glob(os.path.join(TRAIN_DIR, "*.jpeg"))

# Phân loại ảnh
for img_path in train_images:
    filename = os.path.basename(img_path)
    json_path = os.path.join(OUTPUT_JSON_DIR, filename.replace(".jpg", ".json"))

    if os.path.exists(json_path):
        dest_folder = "malignant"  # Có JSON -> malignant
    else:
        dest_folder = "benign"  # Không có JSON -> benign

    shutil.move(img_path, os.path.join(TRAIN_DIR, dest_folder, filename))

print("✅ Phân loại ảnh trong train hoàn tất!")

✅ Phân loại ảnh trong train hoàn tất!


In [8]:
# Thư mục train gốc
TEST_DIR = "/kaggle/working/test"
OUTPUT_JSON_DIR = "/kaggle/working/processed_annotations"

# Tạo thư mục benign/malignant trong train
for folder in ["benign", "malignant"]:
    os.makedirs(os.path.join(TEST_DIR, folder), exist_ok=True)

# Lấy danh sách ảnh trong train
train_images = glob.glob(os.path.join(TEST_DIR, "*.jpg")) + \
               glob.glob(os.path.join(TEST_DIR, "*.jpeg"))

# Phân loại ảnh
for img_path in train_images:
    filename = os.path.basename(img_path)
    json_path = os.path.join(OUTPUT_JSON_DIR, filename.replace(".jpg", ".json"))

    if os.path.exists(json_path):
        dest_folder = "malignant"  # Có JSON -> malignant
    else:
        dest_folder = "benign"  # Không có JSON -> benign

    shutil.move(img_path, os.path.join(TEST_DIR, dest_folder, filename))

print("✅ Phân loại ảnh trong train hoàn tất!")

✅ Phân loại ảnh trong train hoàn tất!


In [9]:
for folder in ["benign", "malignant"]:
    count = len(glob.glob(os.path.join(TRAIN_DIR, folder, "*.jpg")))
    print(f"📂 Số lượng ảnh trong train/{folder}: {count}")

for folder in ["benign", "malignant"]:
    count = len(glob.glob(os.path.join(TEST_DIR, folder, "*.jpg")))
    print(f"📂 Số lượng ảnh trong TEST/{folder}: {count}")

📂 Số lượng ảnh trong train/benign: 1316
📂 Số lượng ảnh trong train/malignant: 1306
📂 Số lượng ảnh trong TEST/benign: 372
📂 Số lượng ảnh trong TEST/malignant: 377


# **Bước 4: Xây dựng hệ thống CNN**

In [10]:
# Định nghĩa ImageDataGenerator cho train + validation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # 🔥 Chia 20% từ train làm validation
)

test_datagen = ImageDataGenerator(rescale=1./255)  # Chỉ cần chuẩn hóa cho test

# Load ảnh từ thư mục train (chia train/val tự động)
train_generator = train_datagen.flow_from_directory(
    "/kaggle/working/train",  
    target_size=(224, 294),
    batch_size=32,
    class_mode="categorical",
    subset="training"  # 🔥 Chỉ lấy phần training (80%)
)

val_generator = train_datagen.flow_from_directory(
    "/kaggle/working/train",
    target_size=(224, 294),
    batch_size=32,
    class_mode="categorical",
    subset="validation"  # 🔥 Chỉ lấy phần validation (20%)
)

# Load ảnh từ thư mục test
test_generator = test_datagen.flow_from_directory(
    "/kaggle/working/test",
    target_size=(224, 294),
    batch_size=32,
    class_mode="categorical",
    shuffle=False  # Không shuffle để đảm bảo đánh giá đúng
)

print("✅ Load dữ liệu hoàn tất!")

Found 2098 images belonging to 2 classes.
Found 524 images belonging to 2 classes.
Found 749 images belonging to 2 classes.
✅ Load dữ liệu hoàn tất!


In [11]:
model = models.Sequential([
    Input(shape=(224, 294, 3)),  
    layers.Conv2D(32, (3,3), activation='relu', padding='same'),
    layers.Conv2D(32, (3,3), activation='relu', padding='same'),
    layers.MaxPooling2D((2,2)),
    layers.Dropout(0.25),

    layers.Conv2D(64, (3,3), activation='relu', padding='same'),
    layers.Conv2D(64, (3,3), activation='relu', padding='same'),
    layers.MaxPooling2D((2,2)),
    layers.Dropout(0.25),

    layers.Conv2D(128, (3,3), activation='relu', padding='same'),
    layers.Conv2D(128, (3,3), activation='relu', padding='same'),
    layers.MaxPooling2D((2,2)),
    layers.Dropout(0.25),

    layers.Conv2D(256, (3,3), activation='relu', padding='same'),
    layers.Conv2D(256, (3,3), activation='relu', padding='same'),
    layers.MaxPooling2D((2,2)),
    layers.Dropout(0.25),

    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(2, activation='softmax')  # Sửa lại số lớp đầu ra
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 224, 294, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 224, 294, 32)        │           9,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 112, 147, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 112, 147, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 112, 147, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 112, 147, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 56, 73, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 56, 73, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 56, 73, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 56, 73, 128)         │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 28, 36, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 28, 36, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_6 (Conv2D)                    │ (None, 28, 36, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_7 (Conv2D)                    │ (None, 28, 36, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 14, 18, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 14, 18, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 64512)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │      33,030,656 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 2)                   │           1,026 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 34,203,938 (130.48 MB)

 Trainable params: 34,203,938 (130.48 MB)

 Non-trainable params: 0 (0.00 B)

# **Huấn luyện mô hình**

In [12]:
model.fit(train_generator,
          epochs=20,
          validation_data=val_generator)

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


66/66 ━━━━━━━━━━━━━━━━━━━━ 96s 971ms/step - accuracy: 0.7837 - loss: 0.3938 - val_accuracy: 0.6088 - val_loss: 0.9986
Epoch 2/20
66/66 ━━━━━━━━━━━━━━━━━━━━ 36s 503ms/step - accuracy: 0.9575 - loss: 0.1609 - val_accuracy: 0.5210 - val_loss: 1.8327
Epoch 3/20
66/66 ━━━━━━━━━━━━━━━━━━━━ 37s 521ms/step - accuracy: 0.9764 - loss: 0.0880 - val_accuracy: 0.9885 - val_loss: 0.0940
Epoch 4/20
66/66 ━━━━━━━━━━━━━━━━━━━━ 37s 522ms/step - accuracy: 0.9769 - loss: 0.0834 - val_accuracy: 0.9637 - val_loss: 0.1897
Epoch 5/20
66/66 ━━━━━━━━━━━━━━━━━━━━ 38s 529ms/step - accuracy: 0.9424 - loss: 0.2893 - val_accuracy: 0.9618 - val_loss: 0.1464
Epoch 6/20
66/66 ━━━━━━━━━━━━━━━━━━━━ 38s 526ms/step - accuracy: 0.9737 - loss: 0.0808 - val_accuracy: 0.9198 - val_loss: 0.1942
Epoch 7/20
66/66 ━━━━━━━━━━━━━━━━━━━━ 37s 519ms/step - accuracy: 0.9816 - loss: 0.0779 - val_accuracy: 0.9866 - val_loss: 0.0836
Epoch 8/20
66/66 ━━━━━━━━━━━━━━━━━━━━ 38s 528ms/step - accuracy: 0.9871 - loss: 0.0382 - val_accuracy: 0.706

In [13]:
# Lưu mô hình hoàn chỉnh (bao gồm cả cấu trúc, trọng số và trạng thái huấn luyện)
model.save('model.h5')

In [14]:
from tensorflow.keras.models import load_model
model = load_model('model.h5')